# Modelo: XGBoost com Tf-Idf

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

In [2]:
def calc_metricas(obs, pred):
    r2 = metrics.r2_score(obs, pred).round(3)
    mape = (100 * metrics.mean_absolute_percentage_error(obs, pred)).round(3)
    mae = metrics.mean_absolute_error(obs, pred).round(2)
    rmse = (metrics.mean_squared_error(obs, pred)**0.5).round(2)
    msle = metrics.mean_squared_log_error(obs, pred).round(3)
    
    return [r2, mape, mae, rmse, msle]

def print_avaliacao(obs, pred):
    print('R² = %.3f' % metrics.r2_score(obs, pred))
    print('MAPE = %.3f %%' % (100 * metrics.mean_absolute_percentage_error(obs, pred)))
    print('MAE = U$S %.2f' % (metrics.mean_absolute_error(obs, pred)))
    print('RMSE = U$S %.2f' % metrics.mean_squared_error(obs, pred)**0.5)
    print('MSLE = %.3f' % metrics.mean_squared_log_error(obs, pred))


In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


def prep_text(text):
    try:
        tokens = word_tokenize(text)
        words = [word.lower() for word in tokens if word.isalpha()]
        words = [word for word in words if not word in stop_words]
    except:
        return None
    return ' '.join(words)

In [4]:
dados_treino = pd.read_csv('../data/processed/train_data.csv')
dados_teste = pd.read_csv('../data/processed/test_data.csv')

## Preparação das colunas de texto

In [7]:
dados_treino.columns

Index(['category_1', 'name', 'item_description', 'price'], dtype='object')

In [8]:
dados_treino = dados_treino[['category_1', 'name', 'item_description', 'price']]
dados_treino.loc[:, 'name'] = dados_treino.apply(lambda x: prep_text(x['name']), axis=1)
dados_treino.loc[:, 'item_description'] = dados_treino.apply(lambda x: prep_text(x['item_description']), axis=1)
dados_treino['comb_name_description'] = dados_treino['name'] + ' ' + dados_treino['item_description']
# dados_treino.dropna(inplace=True)

dados_teste = dados_teste[['category_1', 'name', 'item_description', 'price']]
dados_teste.loc[:, 'name'] = dados_teste.apply(lambda x: prep_text(x['name']), axis=1)
dados_teste.loc[:, 'item_description'] = dados_teste.apply(lambda x: prep_text(x['item_description']), axis=1)
dados_teste['comb_name_description'] = dados_teste['name'] + ' ' + dados_teste['item_description']

## Apenas names como característica

In [ ]:
dados_treino1 = dados_treino[['category_1', 'name', 'price']]
dados_treino1.dropna(inplace=True)

dados_teste1 = dados_teste[['category_1', 'name', 'price']]
dados_teste1.dropna(inplace=True)

In [10]:
pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
X_treino = dados_treino1['name']
y_treino = dados_treino1['price']
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('tiidf',
                 TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2))),
                ('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=101, reg_alpha=0,
                              reg_lambda=1, ...))])

Tempo de treino: 2m 11.3s

In [ ]:
dados_teste1['pred'] = pipe.predict(dados_teste1['name'])
dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo previsão: 11.8s

Avaliação geral

In [12]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.287
MAPE = 89.563 %
MAE = U$S 15.11
RMSE = U$S 32.39
MSLE = 0.489


Avaliação por categoria

In [13]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.293,0.284,0.270,0.367,-0.015,0.117,0.051,0.106,0.117,0.100,0.211
MAPE,74.113,62.731,122.489,116.574,102.788,100.636,123.012,82.317,86.517,165.960,95.879
MAE,14.730,17.690,18.230,23.130,12.400,12.860,14.880,13.340,14.840,14.460,14.830
RMSE,32.890,33.910,43.300,52.340,24.420,20.900,26.720,22.030,27.910,23.790,26.890
MSLE,0.403,0.402,0.692,0.699,0.530,0.516,0.672,0.445,0.494,0.852,0.534


## Modelo por categoria usando apenas a coluna name

In [14]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino1.query(f'category_1 == "{category}"')['name']
    y_treino = dados_treino1.query(f'category_1 == "{category}"')['price']
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


Tempo de treino: 2m 16.2s

In [15]:
dados_teste1 = dados_teste1.assign(pred=0)

In [16]:
for category in dados_teste1.category_1.unique():
    index = dados_teste1.query(f'category_1 == "{category}"').index
    dados_teste1.loc[index, 'pred'] = modelos[category].predict(dados_teste1.loc[index]['name'])

dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo de previsão: 5.7s

Avaliação geral:

In [17]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.341
MAPE = 77.257 %
MAE = U$S 13.83
RMSE = U$S 31.15
MSLE = 0.407


Avaliação por categoria principal

In [18]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.322,0.412,0.223,0.429,0.106,0.288,0.077,0.271,0.321,0.210,-0.031
MAPE,74.436,71.994,111.351,114.824,67.641,68.771,86.106,66.849,70.999,78.811,90.792
MAE,14.490,16.950,17.670,22.150,9.600,10.080,12.020,11.440,12.410,9.670,15.190
RMSE,32.220,30.740,44.690,49.750,22.910,18.760,26.350,19.890,24.470,22.290,30.750
MSLE,0.391,0.378,0.622,0.641,0.343,0.349,0.464,0.347,0.379,0.395,0.521


## Modelo geral usando coluna item_description

In [ ]:
dados_treino1 = dados_treino[['category_1', 'item_description', 'price']]
dados_treino1.dropna(inplace=True)

dados_teste1 = dados_teste[['category_1', 'item_description', 'price']]
dados_teste1.dropna(inplace=True)

In [22]:
pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                 ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
X_treino = dados_treino1['item_description']
y_treino = dados_treino1['price']
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('tiidf',
                 TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2))),
                ('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=101, reg_alpha=0,
                              reg_lambda=1, ...))])

Tempo de treino: 9m 37.8s

In [ ]:
dados_teste1['pred'] = pipe.predict(dados_teste1['item_description'])
dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo de previsão: 20.8s

Avaliação geral

In [26]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.254
MAPE = 91.421 %
MAE = U$S 15.73
RMSE = U$S 33.14
MSLE = 0.510


Avaliação por categoria principal

In [27]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.258,0.204,0.214,0.351,-0.017,0.022,0.098,0.048,0.135,0.107,0.036
MAPE,77.210,67.435,124.147,119.209,103.815,100.666,117.456,83.169,87.665,159.980,94.105
MAE,15.580,18.700,18.620,23.950,12.840,13.000,14.440,13.780,15.080,14.240,15.320
RMSE,33.700,35.760,44.960,53.030,24.440,22.000,26.080,22.740,27.630,23.680,29.730
MSLE,0.439,0.449,0.704,0.715,0.537,0.522,0.639,0.465,0.491,0.822,0.541


## Modelo por categoria usando item_description

In [28]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino1.query(f'category_1 == "{category}"')['item_description']
    y_treino = dados_treino1.query(f'category_1 == "{category}"')['price']
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


Tempo de treino: 7m 18.3s

In [29]:
dados_teste1 = dados_teste1.assign(pred=0)

In [30]:
for category in dados_teste1.category_1.unique():
    index = dados_teste1.query(f'category_1 == "{category}"').index
    dados_teste1.loc[index, 'pred'] = modelos[category].predict(dados_teste1.loc[index]['item_description'])

dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo de previsão: 18.8s

Avaliação geral

In [31]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.293
MAPE = 82.449 %
MAE = U$S 14.79
RMSE = U$S 32.26
MSLE = 0.452


Avaliação por categoria geral

In [32]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.263,0.343,0.140,0.405,0.097,0.186,0.187,0.191,0.229,0.182,-0.086
MAPE,81.633,80.579,113.364,114.074,69.456,74.117,87.382,73.354,75.549,84.407,94.970
MAE,15.730,18.620,18.390,23.020,9.820,10.800,12.140,12.450,13.350,10.290,16.080
RMSE,33.600,32.490,47.020,50.760,23.030,20.070,24.760,20.960,26.080,22.650,31.570
MSLE,0.447,0.452,0.653,0.666,0.361,0.390,0.478,0.394,0.408,0.448,0.564


## Modelo geral usando comb_name_description

In [ ]:
dados_treino1 = dados_treino[['category_1', 'comb_name_description', 'price']]
dados_treino1.dropna(inplace=True)

dados_teste1 = dados_teste[['category_1', 'comb_name_description', 'price']]
dados_teste1.dropna(inplace=True)

In [34]:
pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
X_treino = dados_treino1['comb_name_description']
y_treino = dados_treino1['price']
pipe.fit(X_treino, y_treino)

Pipeline(steps=[('tiidf',
                 TfidfVectorizer(max_df=0.5, min_df=2, ngram_range=(1, 2))),
                ('xgboost',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=0.300000012, max_bin=256,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                              predictor='auto', random_state=101, reg_alpha=0,
                              reg_lambda=1, ...))])

Tempo de treino: 10m 49.4s

In [ ]:
dados_teste1['pred'] = pipe.predict(dados_teste1['comb_name_description'])
dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo previsão: 21.6s

Avaliação geral

In [36]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.349
MAPE = 83.074 %
MAE = U$S 14.44
RMSE = U$S 30.97
MSLE = 0.446


Avaliação por categoria

In [37]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.362,0.352,0.291,0.437,0.027,0.170,0.078,0.125,0.179,0.184,0.209
MAPE,69.137,60.549,115.067,109.730,95.221,91.731,107.649,77.379,79.426,147.984,86.672
MAE,14.090,16.870,17.660,22.100,11.990,12.070,13.740,13.050,14.250,13.300,14.290
RMSE,31.260,32.250,42.690,49.380,23.910,20.270,26.360,21.800,26.910,22.620,26.940
MSLE,0.368,0.372,0.647,0.636,0.488,0.465,0.586,0.421,0.447,0.751,0.486


## Modelo por categoria usando comb_name_description

In [38]:
modelos = {}

for category in dados_treino.category_1.unique():
    pipe = Pipeline([('tiidf', TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range = (1, 2))),
                    ('xgboost', XGBRegressor(random_state=101, n_jobs=-1))])
    modelos[category] = pipe
    X_treino = dados_treino1.query(f'category_1 == "{category}"')['comb_name_description']
    y_treino = dados_treino1.query(f'category_1 == "{category}"')['price']
    # pipe.fit(X_treino, y_treino)
    modelos[category].fit(X_treino, y_treino)


Tempo de treino: 8m 46.4s

In [39]:
dados_teste1 = dados_teste1.assign(pred=0)

In [40]:
for category in dados_teste1.category_1.unique():
    index = dados_teste1.query(f'category_1 == "{category}"').index
    dados_teste1.loc[index, 'pred'] = modelos[category].predict(dados_teste1.loc[index]['comb_name_description'])

dados_teste1.loc[dados_teste1.pred < 3, 'pred'] = 3

Tempo previsão: 25.4s

Avaliação geral

In [41]:
print_avaliacao(dados_teste1['price'], dados_teste1['pred'])

R² = 0.408
MAPE = 71.870 %
MAE = U$S 13.22
RMSE = U$S 29.53
MSLE = 0.374


Avaliação por categoria principal

In [42]:
avaliacoes = {}

for categoria in dados_teste1.category_1.unique():
    dados_cat = dados_teste1.query(f'category_1 == "{categoria}"')
    avaliacoes[categoria] = calc_metricas(dados_cat['price'], dados_cat['pred'])

avaliacoes = pd.DataFrame(avaliacoes, index=['R²', 'MAPE', 'MAE', 'RMSE', 'MSLE'])

avaliacoes

,Women,Men,Vintage & Collectibles,Electronics,Beauty,Kids,Other,Home,Sports & Outdoors,Handmade,No category
R²,0.395,0.454,0.287,0.502,0.162,0.337,0.179,0.314,0.371,0.245,0.084
MAPE,69.517,66.704,102.619,99.942,63.969,65.201,82.076,64.758,66.225,74.057,86.790
MAE,13.860,16.060,17.020,20.650,9.200,9.710,11.700,11.230,12.040,9.460,15.020
RMSE,30.450,29.610,42.810,46.430,22.180,18.110,24.890,19.290,23.570,21.770,28.990
MSLE,0.358,0.345,0.575,0.561,0.319,0.325,0.438,0.332,0.350,0.376,0.514
